**This notebook runs the individual steps of the provided Snakemake pipeline. This may be useful for understanding the functions, but it is highly recommended to use Snakemake to run the pipeline on screening data.**

In [42]:
import ops
from ops.imports_ipython import *
from ops.paper.cell_idr import setup_example

# runs example from repository directory
home = os.path.dirname(os.path.dirname(ops.__file__))
os.chdir(home)

'/Users/sasha/PycharmProjects/OpticalPooledScreens'

In [44]:
os.chdir(os.path.join(home, 'example'))
os.path.join(home, 'example')

'/Users/sasha/PycharmProjects/OpticalPooledScreens/example'

In [45]:
barcodes = pd.read_csv('barcodes.csv')

THRESHOLD_READS = 50
THRESHOLD_DAPI = 200
THRESHOLD_CELL = 600
NUCLEUS_AREA = 40, 400
WILDCARDS = dict(well='A1', tile=102)

SBS_CYCLES = [1, 2, 3, 4, 5, 7, 8, 9, 10]

LUTS = [
    ops.io.GRAY,
    ops.io.GREEN,
    ops.io.RED,
    ops.io.MAGENTA,
    ops.io.CYAN
]

DISPLAY_RANGES = [
    [500, 15000],
    [100, 10000],
    [100, 20000],
    [100, 8000],
    [100, 6000]
]

In [46]:
search = 'experimentC/input/*/10X*{well}_Tile-{tile}.sbs.tif'.format(**WILDCARDS)
input_files = natsorted(glob(search))
for f in input_files:
    print(f)

# used to format output filenames
description = parse(input_files[0])
description['subdir'] = 'experimentC/process_ipynb'
description.pop('cycle');

experimentC/input/stuff/10X_c1-SBS-1_A1_Tile-102.sbs.tif


In [47]:
# ph_search = 'experimentC/input/*/10X*{well}_Tile-{tile}.phenotype.tif'.format(**WILDCARDS)
# ph_input_files = natsorted(glob(ph_search))
# for f in ph_input_files:
#     print(f)

In [48]:
data = np.array([read(f) for f in input_files])
aligned = Snake._align_SBS(data)
save(name(description, tag='aligned'), aligned, display_ranges=DISPLAY_RANGES, luts=LUTS)

In [49]:
# ph_data = read(ph_input_files[0])
# ph_aligned = Snake._align_by_DAPI(data_1=data[0], data_2=ph_data)
# save(name(description, tag='phenotype_aligned'),ph_aligned, luts=LUTS[:2])

In [50]:
loged = Snake._transform_log(aligned, skip_index=0)
save(name(description, tag='log'), loged, display_ranges=DISPLAY_RANGES, luts=LUTS)

In [51]:
maxed = Snake._max_filter(loged, 3, remove_index=0)
save(name(description, tag='maxed'), maxed, display_ranges=DISPLAY_RANGES[1:], luts=LUTS[1:])

### detect candidate reads

In [52]:
std = Snake._compute_std(loged, remove_index=0)
save(name(description, tag='std'), std)

In [53]:
peaks = Snake._find_peaks(std)
save(name(description, tag='peaks'), peaks)

### segment nuclei and cells

In [54]:
nuclei = Snake._segment_nuclei(data[0], THRESHOLD_DAPI,
 area_min=NUCLEUS_AREA[0], area_max=NUCLEUS_AREA[1])

save(name(description, tag='nuclei'), nuclei, compress=1)

In [55]:
cells = Snake._segment_cells(data[0], nuclei, THRESHOLD_CELL)
save(name(description, tag='cells'), cells, compress=1)

### extract base intensity, call reads, assign to cells

In [56]:
df_bases = Snake._extract_bases(maxed, peaks, cells, 
                        THRESHOLD_READS, wildcards=WILDCARDS)
print(df_bases)
df_bases.to_csv(name(description, tag='bases', ext='csv'), index=None)

failed to reshape extracted pixels to sequencing bases, writing empty table


In [16]:
df_reads = Snake._call_reads(df_bases, peaks=peaks)
filename = name(description, tag='reads', ext='csv')
df_reads.to_csv(filename, index=None)

In [17]:
# read from csv to match numerical precision of snakemake pipeline
df_reads = pd.read_csv(filename) 
df_cells = Snake._call_cells(df_reads)
df_cells.to_csv(name(description, tag='cells', ext='csv'), index=None)

### extract phenotypes and combine with called cells

In [18]:
df_phenotype = Snake._extract_named_cell_nucleus_features(
    data=ph_aligned,
    cells=cells,
    nuclei=nuclei,
    nucleus_features=[
        'label', # required to join SBS and phenotype data
        'i',
        'j',
        'area',
        'dapi_gfp_corr',
        'dapi_max',
        'dapi_mean',
        'dapi_median',
        'gfp_max',
        'gfp_mean',
        'gfp_median',
    ],
    cell_features=['label', 'area'],
    wildcards=WILDCARDS
)
df_phenotype.to_csv(name(description, tag='phenotype', ext='csv'), index=None)

In [19]:
df_combined = Snake._merge_sbs_phenotype(
    sbs_tables=df_cells, 
    phenotype_tables=df_phenotype,
    barcode_table=barcodes, 
    sbs_cycles=SBS_CYCLES
)
df_combined.to_csv(name(description, tag='combined', ext='csv'), index=None)

### annotated SBS images

In [20]:
# last channel annotates base calls
annotate_luts = LUTS + [ops.annotate.GRMC, ops.io.GRAY]
annotate_display_ranges = [(a/4, b/4) for a,b in DISPLAY_RANGES] + [[0, 4]]
annotate_SBS = Snake._annotate_SBS(log=loged, df_reads=df_reads)
save(name(description, tag='annotate_SBS'), annotate_SBS,
     display_ranges=annotate_display_ranges, luts=annotate_luts, compress=1)

In [21]:
# second-to-last channel annotates base calls (notches are mapped reads, pluses are unmapped reads)
# last channel encodes peaks value
annotate_extra_luts = LUTS + [ops.annotate.GRMC, ops.io.GRAY, ops.io.GRAY]
annotate_extra_display_ranges = (
    [(a/4, b/4) for a,b in DISPLAY_RANGES]
    +[[0, 4], [0, THRESHOLD_READS*4], [0, 30]]
)
annotate_SBS_extra = Snake._annotate_SBS_extra(
    log=loged,
    peaks=peaks,
    df_reads=df_reads,
    barcode_table=barcodes,
    sbs_cycles=SBS_CYCLES
)
save(name(description, tag='annotate_SBS_extra'), annotate_SBS_extra,
     display_ranges=annotate_extra_display_ranges[1:], luts=annotate_extra_luts[1:], compress=1)